1. setup

Importera in bibliotek
Sätt k-värde 
Sät top_n-värde

In [70]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split



Nedan lägger vi till csv-filerna med Användares betyg och spelen 
....

In [104]:
df_ratings = pd.read_csv("dataset/user_ratings.csv")
df_games = pd.read_csv("dataset/games.csv")
df_ratings.info()
#df_games.info()
df = pd.merge(df_ratings, df_games[['BGGId', 'Name']], on='BGGId', how='left')
df.info()   


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18942215 entries, 0 to 18942214
Data columns (total 3 columns):
 #   Column    Dtype  
---  ------    -----  
 0   BGGId     int64  
 1   Rating    float64
 2   Username  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 433.6+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 18942215 entries, 0 to 18942214
Data columns (total 4 columns):
 #   Column    Dtype  
---  ------    -----  
 0   BGGId     int64  
 1   Rating    float64
 2   Username  object 
 3   Name      object 
dtypes: float64(1), int64(1), object(2)
memory usage: 722.6+ MB


In [105]:
df_ratings

,BGGId,Rating,Username
0,213788,8.0,Tonydorrf
1,213788,8.0,tachyon14k
2,213788,8.0,Ungotter
3,213788,8.0,brainlocki3
4,213788,8.0,PPMP
...,...,...,...
18942210,165521,3.0,rseater
18942211,165521,3.0,Bluefox86
18942212,165521,3.0,serginator
18942213,193488,1.0,CaptainCattan


In [106]:
df.describe()

,BGGId,Rating
count,1.894222e+07,1.894222e+07
mean,1.099533e+05,7.130957e+00
std,9.280217e+04,1.543023e+00
min,1.000000e+00,1.000000e-04
25%,1.611000e+04,6.000000e+00
50%,1.075290e+05,7.000000e+00
75%,1.813040e+05,8.000000e+00
max,3.491610e+05,1.000000e+01


2 Preprocessing

Handle missing valuee

In [107]:
any_null_values = df.isnull().any()
any_null_values

null_counts = df.isnull().sum()
print(null_counts)

BGGId        0
Rating       0
Username    63
Name         0
dtype: int64


In [108]:
df = df.dropna()

In [109]:
df.isnull().any()

BGGId       False
Rating      False
Username    False
Name        False
dtype: bool

In [110]:
df

,BGGId,Rating,Username,Name
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition
...,...,...,...,...
18942210,165521,3.0,rseater,Rory's Story Cubes: Medic
18942211,165521,3.0,Bluefox86,Rory's Story Cubes: Medic
18942212,165521,3.0,serginator,Rory's Story Cubes: Medic
18942213,193488,1.0,CaptainCattan,Airlines


In [111]:
duplicates = df.duplicated().value_counts()
duplicates

False    18942152
dtype: int64

In [112]:
duplicates.describe()

count           1.0
mean     18942152.0
std             NaN
min      18942152.0
25%      18942152.0
50%      18942152.0
75%      18942152.0
max      18942152.0
dtype: float64

Remove duplicate ratings for the same game by the same user (due to different version of the game etc)

In [113]:
# Calculate the mean rating for each group of duplicate rows
mean_ratings = df.groupby(['Username', 'Name'])['Rating'].mean()

In [114]:
mean_ratings

Username        Name                               
 Fu_Koios       Approaching Dawn: The Witching Hour     9.0
                Shadowrift                              9.0
 beastvol       Candamir: The First Settlers            5.0
                Catan                                   8.0
                Catan Card Game                         7.0
                                                       ... 
Æleksandr Þræð  Mahjong                                10.0
                Mille Bornes                            7.0
                Senet                                   7.0
                UNO                                     8.0
                Vampire                                 7.0
Name: Rating, Length: 18884652, dtype: float64

In [119]:
# Filter the DataFrame for the specific user and item
specific_user_item_ratings = df[(df['Username'] == 'hariseldoncwb') & (df['BGGId'] == 135382)]

# Calculate the mean rating for the specific user and item
mean_rating_specific_user_item = specific_user_item_ratings['Rating'].mean()
mean_rating_specific_user_item

5.5

In [120]:
# Merge the mean ratings back to the original DataFrame based on 'Username' and 'Name'
df = df.merge(mean_ratings, on=['Username', 'Name'], suffixes=('', '_mean'))

In [121]:
df

,BGGId,Rating,Username,Name,Rating_mean
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition,8.0
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition,8.0
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition,8.0
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition,8.0
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition,8.0
...,...,...,...,...,...
18942147,165521,3.0,rseater,Rory's Story Cubes: Medic,3.0
18942148,165521,3.0,Bluefox86,Rory's Story Cubes: Medic,3.0
18942149,165521,3.0,serginator,Rory's Story Cubes: Medic,3.0
18942150,193488,1.0,CaptainCattan,Airlines,1.0


In [122]:
# Filter the DataFrame to keep only the original rows (non-duplicates) and the calculated mean ratings
df_unique = df[~df.duplicated(subset=['Username', 'Name'], keep=False)]

In [123]:
df_unique

,BGGId,Rating,Username,Name,Rating_mean
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition,8.0
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition,8.0
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition,8.0
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition,8.0
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition,8.0
...,...,...,...,...,...
18942147,165521,3.0,rseater,Rory's Story Cubes: Medic,3.0
18942148,165521,3.0,Bluefox86,Rory's Story Cubes: Medic,3.0
18942149,165521,3.0,serginator,Rory's Story Cubes: Medic,3.0
18942150,193488,1.0,CaptainCattan,Airlines,1.0


In [124]:
# Drop the original 'Rating' column and rename the mean rating column
df = df_unique.drop(columns=['Rating']).rename(columns={'Rating_mean': 'Rating'})

In [128]:
# Reindex the columns to move 'Rating' to the second position
column_order = ['BGGId', 'Rating', 'Username', 'Name']
df = df.reindex(columns=column_order)

In [129]:
df

,BGGId,Rating,Username,Name
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition
...,...,...,...,...
18942147,165521,3.0,rseater,Rory's Story Cubes: Medic
18942148,165521,3.0,Bluefox86,Rory's Story Cubes: Medic
18942149,165521,3.0,serginator,Rory's Story Cubes: Medic
18942150,193488,1.0,CaptainCattan,Airlines


In [130]:
# Group by 'BGGId' and count the number of rows for each group
grouped_counts = df.groupby('BGGId').size()
grouped_counts

BGGId
1          5295
2           560
3         14969
4           342
5         18152
          ...  
347146       42
347521       51
348955       67
349131       66
349161        7
Length: 21925, dtype: int64

Filter dataset

In [86]:
game_ratings = 50

rating_counter = df['BGGId'].value_counts()

filtered_out = rating_counter[rating_counter < game_ratings].index

games_mask = df['BGGId'].isin(filtered_out)
#games_mask = df['BGGId'].isin(rating_counter[rating_counter < game_ratings].index)
#games_mask
df.drop(index=df[games_mask].index, inplace=True)

df

,BGGId,Rating,Username,Name
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition
...,...,...,...,...
18942210,165521,3.0,rseater,Rory's Story Cubes: Medic
18942211,165521,3.0,Bluefox86,Rory's Story Cubes: Medic
18942212,165521,3.0,serginator,Rory's Story Cubes: Medic
18942213,193488,1.0,CaptainCattan,Airlines


In [87]:
# Group by 'BGGId' and count the number of rows for each group
grouped_counts = df.groupby('BGGId').size()
grouped_counts

BGGId
1          5352
2           562
3         15147
4           342
5         18387
          ...  
346703     1000
346965      253
347521       71
348955       67
349131       66
Length: 17552, dtype: int64

In [88]:
# Group by 'Username' and count the number of rows for each group
grouped_user_counts = df.groupby('Username').size()
grouped_user_counts

Username
 Fu_Koios              2
 beastvol              9
 mycroft              14
 woh                   5
(mostly) harmless      1
                    ... 
zzzuzu                39
zzzvone               21
zzzxxxyyy             36
zzzzzane             154
Æleksandr Þræð        12
Length: 408188, dtype: int64

In [89]:
user_ratings = 20

rating_counter = df['Username'].value_counts()

filtered_out = rating_counter[rating_counter < user_ratings].index

user_mask = df['Username'].isin(filtered_out)

df.drop(index=df[user_mask].index, inplace=True)

df

,BGGId,Rating,Username,Name
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition
...,...,...,...,...
18942209,165521,4.0,crito13,Rory's Story Cubes: Medic
18942210,165521,3.0,rseater,Rory's Story Cubes: Medic
18942211,165521,3.0,Bluefox86,Rory's Story Cubes: Medic
18942212,165521,3.0,serginator,Rory's Story Cubes: Medic


In [90]:
# Group by 'Username' and count the number of rows for each group
grouped_user_counts = df.groupby('Username').size()
grouped_user_counts

Username
-=Yod@=-       304
-Johnny-       478
-Loren-         79
-LucaS-         23
-Morphling-     51
              ... 
zzzoren        184
zzzuzu          39
zzzvone         21
zzzxxxyyy       36
zzzzzane       154
Length: 168446, dtype: int64

Train test split

In [95]:
# Define a function to split the data for each user
def user_split(data, test_size=0.2):    
    train_to_concat = []
    test_to_concat = []
    for user, group in data.groupby('Username'):
        train_user, test_user = train_test_split(group, test_size=test_size, train_size=1-test_size)
        train_to_concat.append(train_user)
        test_to_concat.append(test_user)
    
    train = pd.concat(train_to_concat, ignore_index=True)
    test = pd.concat(test_to_concat, ignore_index=True)

    return train, test

# Split the data
train_set, test_set = user_split(df, test_size=0.2)


In [96]:
train_set

,BGGId,Rating,Username,Name
0,93,8.0,-=Yod@=-,El Grande
1,216091,7.0,-=Yod@=-,Unlock!: Escape Adventures – The Formula
2,7,7.5,-=Yod@=-,Cathedral
3,279613,8.0,-=Yod@=-,Unlock!: Timeless Adventures
4,503,7.0,-=Yod@=-,Through the Desert
...,...,...,...,...
13875454,822,7.0,zzzzzane,Carcassonne
13875455,96848,5.0,zzzzzane,Mage Knight Board Game
13875456,136063,7.0,zzzzzane,Forbidden Desert
13875457,161970,8.0,zzzzzane,Alchemists


In [97]:
test_set

,BGGId,Rating,Username,Name
0,228867,8.0,-=Yod@=-,Unlock!: Mystery Adventures
1,147020,8.0,-=Yod@=-,Star Realms
2,25821,7.0,-=Yod@=-,The Werewolves of Miller's Hollow
3,31260,7.5,-=Yod@=-,Agricola
4,80006,7.0,-=Yod@=-,Mord im Arosa
...,...,...,...,...
3553002,230802,8.0,zzzzzane,Azul
3553003,98778,7.0,zzzzzane,Hanabi
3553004,168435,9.0,zzzzzane,Between Two Cities
3553005,20100,8.0,zzzzzane,Wits & Wagers


In [99]:
# Find duplicate rows based on 'Username' and 'Name'
duplicate_rows = df[df.duplicated(subset=['Username', 'Name'], keep=False)]

In [100]:
duplicate_rows

,BGGId,Rating,Username,Name
339,126021,6.9,grimsmirk,Dropzone Commander
379,158968,3.0,hariseldoncwb,Maha Yodha
662,165283,3.0,duffee,Battleship
667,165283,2.0,dazel97,Battleship
669,165283,1.0,Irondell,Battleship
...,...,...,...,...
18938870,91666,7.0,schlawiner,Crossboule
18938907,91666,6.5,howea,Crossboule
18939388,166026,4.0,Tevmyrn,Munchkin Treasure Hunt
18939470,111866,6.0,PSchulman,Rise!


In [103]:
duplicate_rows[duplicate_rows['Username'] == 'hariseldoncwb']

,BGGId,Rating,Username,Name
379,158968,3.0,hariseldoncwb,Maha Yodha
45661,158968,5.0,hariseldoncwb,Maha Yodha
9086366,4143,4.0,hariseldoncwb,Guess Who?
9681257,4143,4.5,hariseldoncwb,Guess Who?
12864936,135382,4.0,hariseldoncwb,Shadowrun: Crossfire
13193748,135382,7.0,hariseldoncwb,Shadowrun: Crossfire


filtrera dataframe 

3 Similarity Calculation

 Skapa en matris av item-för-item-similarity baserat på cosine similarity

In [98]:
# Assuming df is your pandas DataFrame with columns BGGId, Rating, Username, Name
# Pivot the DataFrame so that rows represent users and columns represent items
pivot_df = df.pivot(index='Username', columns='Name', values='Rating').fillna(0)



/var/folders/f3/n44hzk017jg3wr6ylhv9l8lr0000gn/T/ipykernel_91836/2345889906.py:3: PerformanceWarning: The following operation may generate 2907883298 cells in the resulting pandas object.
  pivot_df = df.pivot(index='Username', columns='Name', values='Rating').fillna(0)


ValueError: Index contains duplicate entries, cannot reshape

In [ ]:
# Compute cosine similarity between items
item_similarity = cosine_similarity(pivot_df.T)  # Transpose the pivot DataFrame to get item-item similarity

# Convert item similarity array to DataFrame for easier manipulation
item_similarity_df = pd.DataFrame(item_similarity, index=pivot_df.columns, columns=pivot_df.columns)

Här nedan skriver vi funktionen som tar input data, user, k, top_n

Nedan skriver vi funktion som visar resultat av rekomendation i form av en sorts lista baserat på recommender funktionen

Nedan här skriver vi evalueringsfunktioner av modellen och utskrift 